In [1]:
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import swifter


# Подготовка датасета для загрузки в сеть

In [2]:
base_df = pd.read_csv('data/clean_frame.csv', index_col='ind')
base_df.columns

Index(['ID', 'Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Пособие',
       'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Опекунство', 'Село',
       'Иностранец', 'КодФакультета', 'СрБаллАттестата', 'Статус', 'male',
       'female', 'sex_summ', 'birth_year_int', 'basis', 'language', 'country',
       'region', 'city', 'parents_country'],
      dtype='object')

### Проверка и удаление отрицательных значений

In [3]:
base_df.isna().any().all()

False

In [5]:
base_df = base_df.fillna(0)

In [ ]:
drop_col = ['Статус']

In [6]:
def get_accuracy(frame_to_train):
    train, test = train_test_split(frame_to_train, test_size=0.20)
    
    train_input = train.drop(drop_col, axis=1)    
    train_output = train['Статус']

    test_input = test.drop(drop_col, axis=1)
    test_output = test['Статус']

    clf = RandomForestClassifier(random_state=0)
    clf.fit(train_input, train_output)
    pred = clf.predict(test_input)
    return f1_score(test_output, pred, average='macro', zero_division = 0)

In [7]:
all_columns_list = base_df.columns.to_list()
all_columns_list.remove('ID')
all_columns_list.remove('Статус')
all_columns_list

['Код_группы',
 'Год_Поступления',
 'Год_Окончания_УЗ',
 'Пособие',
 'Общежитие',
 'Наличие_Матери',
 'Наличие_Отца',
 'Опекунство',
 'Село',
 'Иностранец',
 'КодФакультета',
 'СрБаллАттестата',
 'male',
 'female',
 'sex_summ',
 'birth_year_int',
 'basis',
 'language',
 'country',
 'region',
 'city',
 'parents_country']

In [ ]:
bad_columns = []

In [13]:
elements_num = len(all_columns_list)
elements_num = 6

In [14]:
all_combinations = []
list_sizes = 0
min_col = 4
for i in range(min_col, elements_num+1):
    curent_list = list(combinations(all_columns_list, i))
    list_sizes += len(curent_list)
    all_combinations = all_combinations + curent_list
comb_size = len(all_combinations)
comb_size

108262

In [15]:
all_combinations

[('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Пособие'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Общежитие'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Наличие_Матери'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Наличие_Отца'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Опекунство'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Село'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Иностранец'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'КодФакультета'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'СрБаллАттестата'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'male'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'female'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'sex_summ'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'birth_year_int'),
 ('Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'basis'),
 ('Код_группы', 'Год_Поступле

In [16]:
combinations_frame = pd.DataFrame(all_combinations, columns=range(elements_num))
combinations_frame


,0,1,2,3,4,5
0,Код_группы,Год_Поступления,Год_Окончания_УЗ,Пособие,None,None
1,Код_группы,Год_Поступления,Год_Окончания_УЗ,Общежитие,None,None
2,Код_группы,Год_Поступления,Год_Окончания_УЗ,Наличие_Матери,None,None
3,Код_группы,Год_Поступления,Год_Окончания_УЗ,Наличие_Отца,None,None
4,Код_группы,Год_Поступления,Год_Окончания_УЗ,Опекунство,None,None
...,...,...,...,...,...,...
108257,birth_year_int,basis,language,country,city,parents_country
108258,birth_year_int,basis,language,region,city,parents_country
108259,birth_year_int,basis,country,region,city,parents_country
108260,birth_year_int,language,country,region,city,parents_country


In [17]:
def itter_from_frame(row):
    frame_columns = row.dropna().to_list() + ['Статус']
    frame_to_train = base_df.loc[:, frame_columns]
    return get_accuracy(frame_to_train)

In [18]:
start, stop = 0, comb_size
combinations_frame.loc[start:stop, 'accuracy'] = combinations_frame.iloc[start:stop, :].apply(itter_from_frame, axis=1)

In [19]:
combinations_frame.head(30)

,0,1,2,3,4,5,accuracy
0,Код_группы,Год_Поступления,Год_Окончания_УЗ,Пособие,None,None,0.692915
1,Код_группы,Год_Поступления,Год_Окончания_УЗ,Общежитие,None,None,0.710932
2,Код_группы,Год_Поступления,Год_Окончания_УЗ,Наличие_Матери,None,None,0.712926
3,Код_группы,Год_Поступления,Год_Окончания_УЗ,Наличие_Отца,None,None,0.715503
4,Код_группы,Год_Поступления,Год_Окончания_УЗ,Опекунство,None,None,0.714770
5,Код_группы,Год_Поступления,Год_Окончания_УЗ,Село,None,None,0.705707
6,Код_группы,Год_Поступления,Год_Окончания_УЗ,Иностранец,None,None,0.730142
7,Код_группы,Год_Поступления,Год_Окончания_УЗ,КодФакультета,None,None,0.729686
8,Код_группы,Год_Поступления,Год_Окончания_УЗ,СрБаллАттестата,None,None,0.690537
9,Код_группы,Год_Поступления,Год_Окончания_УЗ,male,None,None,0.700919


In [21]:
combinations_frame['accuracy'].describe()

count    108262.000000
mean          0.489459
std           0.133048
min           0.246111
25%           0.393522
50%           0.444209
75%           0.638015
max           0.801658
Name: accuracy, dtype: float64

In [20]:
rt = combinations_frame.sort_values('accuracy', ascending=False)
rt.to_csv(f'data/combinations/columns_from_{min_col}_to_{elements_num}.csv')


In [97]:
rt.groupby('accuracy').describe()

0                                 1                            
         count unique             top freq count unique             top freq
accuracy                                                                    
0.247861     1      1          female    1     1      1        language    1
0.248441     2      2  Наличие_Матери    1     2      2    Наличие_Отца    1
0.249309     1      1  Наличие_Матери    1     1      1        sex_summ    1
0.249405     1      1         Пособие    1     1      1          female    1
0.249532     1      1         Пособие    1     1      1      Опекунство    1
...        ...    ...             ...  ...   ...    ...             ...  ...
0.751500     1      1      Код_группы    1     1      1         Пособие    1
0.754032     1      1      Код_группы    1     1      1   КодФакультета    1
0.754564     1      1      Код_группы    1     1      1    Наличие_Отца    1
0.760877     1      1      Код_группы    1     1      1  Наличие_Матери    1
0.763686     1      1      Код_группы    1     1      1      Опекунство    1

[223 rows x 8 columns]

In [ ]:
accuracy = {}
for num, columns in enumerate(all_combinations[:100]):
    frame_columns = list(columns)
    frame_columns.append('Статус')
    frame_to_train = base_df.loc[:, frame_columns]
    accuracy[num] = get_accuracy(frame_to_train)
    print(num)
accuracy


In [ ]:
accuracy

In [ ]:
max_index = max(accuracy, key=accuracy.get)
print(max_index, all_combinations[max_index], accuracy[max_index])